In [1]:
from selenium import webdriver
from bs4  import BeautifulSoup
import requests
import tqdm
import pandas as pd
import time

import os
import re

In [22]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(r"C:\Users\User\Downloads\Compressed\chromedriver_win32_2\chromedriver.exe",options=chrome_options)

srch_url="https://www.flipkart.com/womens-dresses/pr?sid=clo%2Codx%2Cmaj%2Cjhy&marketplace=FLIPKART&otracker=product_breadCrumbs_Women%27s+Dresses"


In [23]:
re.sub('\W+',' ',srch_url )[:20]

'https www flipkart c'

In [24]:
def down_img(url,scd):
    img_data = requests.get(url).content
    if not os.path.exists('downloaded_image'):
        os.makedirs('downloaded_image')
    else:
        img_name=os.getcwd()+'/downloaded_image'+'/%0.f_SC_%s.jpg'%(time.time(),re.sub('\W+',' ',scd )[:20])
        with open(img_name, 'wb') as handler:
            handler.write(img_data)
#             print(img_name+'  Image downloaded')
        return './downloaded_image'+'/%0.f_SC_%s.jpg'%(time.time(),re.sub('\W+',' ',scd ))

In [25]:
class scrape_one_prod:
#     global otdic
    def __init__(self):
        self.name= list()
        self.price = list()
        self.dis = list()
        self.url = []
        self.df_p= pd.DataFrame()
        
    def scrape_details(self,srch_url,pg_no):
        driver.get(srch_url+r'&page='+str(pg_no))
        content = driver.page_source
        soup = BeautifulSoup(content)
        error_url = list()
        for i,a in enumerate(tqdm.tqdm(soup.findAll(attrs={'class':'_2B099V'}))):
            
            try: 
                a.find('a',attrs='IRpwTa')['href']
                self.url='https://www.flipkart.com' + a.find('a',attrs='IRpwTa')['href']
#                 url.append(+self.url)
                self.scrape_product(self.url,a)
            except TypeError:
                error_url.append('https://www.flipkart.com' + a.find('a',attrs='IRpwTa')['href'])
                print('Error in fetching product trying to fetch next product in 5sec')
                time.sleep(5)
                
        return error_url
    
    def scrape_product(self,url_,a):
#         print(url_)
        driver.get(url_)
        content_=driver.page_source
        soup = BeautifulSoup(content_)
        otdic = dict()
        dec,colr_,col,siz = str(),str(),list(),list()
        
        #Name Product color rating and product price
        self.name=a.find('a',attrs='IRpwTa')
        self.dis=a.find('div',attrs='_3Ay6Sb')
        self.price=a.find('div',attrs='_30jeq3')
        rat = str()
        if soup.find(attrs={'class':'_3LWZlK _3uSWvT'}): #soup.find(attrs={'class':'_3LWZlK _3uSWvT'}).text
            rat = soup.find(attrs={'class':'_3LWZlK _3uSWvT'}).text
#             print(rat)
        else:
            rat =None
        
        otdic.update({'Product_name':self.name,'Price':self.price,'Didcount':self.dis,'P_URL':url_,'Rating':rat})
        
        #Description
        if soup.find('div',attrs='_1AN87F'):
            dec = soup.find('div',attrs='_1AN87F').text
#             otdic.update({'Description':dec})
#             print('Product Decription found :'+otdic['Description']
        else:
            dec = None
#             otdic.update({'Description':'None'})
            
        #Product Color    
        if soup.find('li',attrs={'id':re.compile('-color')}):
            for itm in soup.find_all('li','_3V2wfe _31hAvz'):
                if itm.find('a','kmlXmn PP89tw'):
                    colr_ = itm.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text
#                     otdic.update({'P_Color':colr_})
#                     print('Product Color :'+otdic['P_Color'])
        else:
            colr_=None
#             otdic.update({'P_Color':'None'})
            
         
        #Available colors
        if soup.find('li',{'id':re.compile('-color')}):
            for items in soup.find_all('li',{'class':'_3V2wfe _31hAvz','id':re.compile('-color')}):
                col.append(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)
            #     print(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)
#             otdic.update({'Avl_Color':col})
#             print('Available COlors ',end='\t')
#             print(otdic['Avl_Color'])
        else:
            col =[None]
#             otdic.update({'Avl_Color':'None'})
        
        #Available sizes
        if soup.find('li',{'id':re.compile('-size')}):
            for items in soup.find_all('li',{'class':'_3V2wfe _31hAvz','id':re.compile('-size')}):
                siz.append(items.find('div',attrs={'class':re.compile('_3Oikkn _3_ezix _2KarXJ _31hAvz')}).text)
            #     print(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)            
#             otdic.update({'Avl_Size':siz})
#             print('Available Size ',end='\t')
#             print(otdic['Avl_Size']) 
        else:
            siz = [None]
#             otdic.update({'Avl_Size':'None'})
        
        #Image URL
        img_url_ = soup.find('img','_2r_T1I _396QI4')['src']
        
#         Other details of product
        ot_ky, ky_vl = [],[]
        for k,v in zip(soup.find_all('div',attrs=['_2H87wv',]), soup.find_all('div',attrs=['_2vZqPX'])):
#             print(k.text+': '+v.text)
#             otdic.update({k.text:v.text})
            ot_ky.append(k.text)
            ky_vl.append(v.text)
        otdic.update({k__:v__ for k__, v__ in zip(ot_ky,ky_vl)})
#         print(otdic)
        im_path = down_img(img_url_,otdic['Style Code'])
        otdic.update({'Description': dec, 'P_Color':colr_ , 'Avl_Color': [col], 'Avl_Size':[siz] ,'Image_path':im_path})
        self.df_p = pd.DataFrame.from_dict(otdic)
        global df
        df = pd.concat([df,self.df_p])
#         return otdic
#         pass
        

In [26]:
global df
df = pd.DataFrame()
scrapper = scrape_one_prod()

In [28]:
pg_no = 25

for i in range(1,pg_no):
    error_url_list = scrapper.scrape_details(srch_url,i)
#     print(srch_url+str(i))
#     dic = {'Product Name':name,'Price':price,'Rating':dis,'url':url}
#     dic.update
#     df = df.append(pd.DataFrame(ot_dic),ignore_index=True)
    print('Extracted %d Page products and total %d products '%(i,df.shape[0]))

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:49<00:00,  2.73s/it]


Extracted 1 Page products and total 40 products 


 60%|█████████████████████████████████████████████████▏                                | 24/40 [00:55<00:38,  2.42s/it]

Error in fetching product trying to fetch next product in 5sec


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:47<00:00,  2.68s/it]


...

Extracted 22 Page products and total 877 products 


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [02:40<00:00,  4.01s/it]


Extracted 23 Page products and total 917 products 


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [02:38<00:00,  3.97s/it]

Extracted 24 Page products and total 957 products 


In [29]:
df.to_csv('extracted_%0.f.csv'%time.time())

In [47]:
def get_dupli(col,valu):
    vc = df[col].value_counts()
    return len(vc> valu)
    

In [48]:
get_dupli('P_URL',1)

664

In [43]:
vc  = df['Image_path'].value_counts() 

In [46]:
len(vc[vc > 1])

178